In [1]:
import pandas as pd

In [6]:
annual_gdp = pd.read_csv('../data/annual_gdp.csv')
annual_unemployment = pd.read_csv('../data/annual_unemployment.csv')
quarterly_gdp = pd.read_csv('../data/quarterly_gdp.csv')
quarterly_unemployment = pd.read_csv('../data/quarterly_unemployment.csv')

In [7]:
annual_gdp

,DATE,GDPC1
0,1960-01-01,14001.090
1,1961-01-01,14360.265
2,1962-01-01,15240.493
3,1963-01-01,15904.567
4,1964-01-01,16821.109
...,...,...
59,2019-01-01,82862.686
60,2020-01-01,81070.339
61,2021-01-01,85979.193
62,2022-01-01,88139.315
